In [ ]:
import pluto_sdr_pr.processing

from scipy.signal import hilbert

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
duration = 1
fs = 1000
samples = int(fs*duration)
t = np.arange(samples) / fs
pulse_on_duration = 50
pulse_off_duration = samples - pulse_on_duration
time_offset = 250
freq_offset = 120

waveform = np.concatenate([np.sin(t[:pulse_on_duration] * 100 * 2 * np.pi), np.zeros(pulse_off_duration)])
ref_waveform = waveform
ref_waveform += np.random.default_rng(seed=1337).standard_normal(samples) * 0.2
surv_waveform = 0.1 * np.roll(np.real(waveform * np.exp(-2j * t * np.pi * freq_offset)), time_offset)
surv_waveform += ref_waveform * 0.5
surv_waveform += np.random.default_rng(seed=1337).standard_normal(samples) * 0.01

ref_signal = hilbert(ref_waveform)
surv_signal = hilbert(surv_waveform)

amb = pluto_sdr_pr.processing.fast_ambiguity(samples, fs, ref_signal, surv_signal)
peak = np.unravel_index(np.argmax(np.abs(amb)), amb.shape)

fig, (ax0, ax1, ax2, ax3) = plt.subplots(4, figsize=(25, 10))
fig.suptitle("Reference and Echo Signals as Recorded by Receiver", fontsize=16)

ax0.plot(waveform)
ax0.set_xlabel("samples")
ax0.set_xlim([0, t.shape[0]])
ax0.set_ylim([-1, 1])
ax0.set_title("Sent Waveform")

ax1.plot(ref_waveform / np.abs(ref_waveform).max())
ax1.set_xlabel("samples")
ax1.set_xlim([0, t.shape[0]])
ax1.set_ylim([-1, 1])
ax1.set_title("Noisy Reference Receiver Channel")

ax2.plot(surv_waveform / np.abs(ref_waveform).max())
ax2.set_xlabel("samples")
ax2.set_xlim([0, t.shape[0]])
ax2.set_ylim([-1, 1])
ax2.set_title("Noisy Surveillance Receiver Channel")
ax2.axvspan(0, pulse_on_duration, facecolor='red', alpha=.2)
ax2.axvspan(time_offset, time_offset + pulse_on_duration, facecolor='blue', alpha=.2)

ax3.imshow(np.abs(amb.T), interpolation="nearest", aspect="auto", cmap="Greys")
ax3.set_yticks(np.linspace(0, fs, 9, endpoint=True))
ax3.set_yticklabels(
    map(lambda y: f"{-(y - fs // 2):.0f}", ax3.get_yticks())
)
ax3.set_xlabel("range")
ax3.set_ylabel("doppler")
ax3.set_xlim([0, t.shape[0]])
ax3.set_ylim([fs, 0])
ax3.set_title("Range/Doppler Map")
_ = ax3.annotate(
    f"Peak ({peak[0]},{-(peak[1] - fs // 2):.0f})",
    peak,   
    xytext=(10, -20),
    xycoords="data",
    textcoords="offset pixels",
    arrowprops={"arrowstyle": "wedge"},
)

fig.tight_layout()

In [ ]:
def ambfun(ref, surv):
    return pluto_sdr_pr.processing.fast_ambiguity(samples, fs, ref, surv)

amb_pre_clean = pluto_sdr_pr.processing.fast_ambiguity(
    samples,
    fs,
    ref_signal,
    surv_signal,
)
abs_amb_pre_clean = np.abs(amb_pre_clean)

cleaned_surv_signal, _ = pluto_sdr_pr.processing.clean(ref_signal, surv_signal, ambfun)

amb_post_clean = pluto_sdr_pr.processing.fast_ambiguity(
    samples,
    fs,
    ref_signal,
    cleaned_surv_signal,
)
abs_amb_post_clean = np.abs(amb_post_clean)
peak = np.unravel_index(np.argmax(np.abs(amb_post_clean)), amb_post_clean.shape)

x = np.arange(amb_pre_clean.shape[0])
y = np.arange(amb_pre_clean.shape[1])
X, Y = np.meshgrid(x, y) 

fig = plt.figure(figsize=(25, 10))
fig.suptitle("Application of CLEAN Algorithm", fontsize=16)

ax = fig.add_subplot(2, 3, (1, 2))
ax.imshow(abs_amb_pre_clean.T, interpolation="nearest", aspect="auto", cmap="Greys")
ax.set_yticks(np.linspace(0, fs, 9, endpoint=True))
ax.set_yticklabels(
    map(lambda y: f"{-(y - fs // 2):.0f}", ax.get_yticks())
)
ax.set_xlabel("range")
ax.set_ylabel("doppler")
ax.set_xlim([0, t.shape[0]])
ax.set_ylim([fs, 0])
ax.set_title("Range/Doppler Map before CLEAN")

ax = fig.add_subplot(2, 3, 3, projection="3d")
ax.plot_surface(X, Y, abs_amb_pre_clean.T / abs_amb_pre_clean.max(), cmap="inferno")
ax.set_yticks(np.linspace(0, fs, 5, endpoint=True))
ax.set_yticklabels(
    map(lambda y: f"{-(y - fs // 2):.0f}", ax.get_yticks())
)
ax.set_xlabel("range")
ax.set_ylabel("doppler")
ax.set_xlim([0, t.shape[0]])
ax.set_ylim([fs, 0])
ax.set_title("Range/Doppler Map after CLEAN")

ax = fig.add_subplot(2, 3, (4, 5))
ax.imshow(abs_amb_post_clean.T, interpolation="nearest", aspect="auto", cmap="Greys")
ax.set_yticks(np.linspace(0, fs, 9, endpoint=True))
ax.set_yticklabels(
    map(lambda y: f"{-(y - fs // 2):.0f}", ax.get_yticks())
)
ax.set_xlabel("range")
ax.set_ylabel("doppler")
ax.set_xlim([0, t.shape[0]])
ax.set_ylim([fs, 0])
ax.set_title("Range/Doppler Map after CLEAN")
_ = ax.annotate(
    f"Peak ({peak[0]},{-(peak[1] - fs // 2):.0f})",
    peak,   
    xytext=(10, -20),
    xycoords="data",
    textcoords="offset pixels",
    arrowprops={"arrowstyle": "wedge"},
)

ax = fig.add_subplot(2, 3, 6, projection="3d")
ax.plot_surface(X, Y, abs_amb_post_clean.T / abs_amb_post_clean.max(), cmap="inferno")
ax.set_yticks(np.linspace(0, fs, 5, endpoint=True))
ax.set_yticklabels(
    map(lambda y: f"{-(y - fs // 2):.0f}", ax.get_yticks())
)
ax.set_xlabel("range")
ax.set_ylabel("doppler")
ax.set_xlim([0, t.shape[0]])
ax.set_ylim([fs, 0])
ax.set_title("Range/Doppler Map after CLEAN")

print()